In [1]:
!pip install kaggle

In [2]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"tarunpaparaju","key":"6ff13d7192c816d306273588a9bb2dab"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [5]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [6]:
!kaggle competitions download -c quora-insincere-questions-classification -p /content

embeddings.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
# Install PyTorch

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
  
# Install pytorch-pretrained-BERT

!pip install pytorch-pretrained-bert

# Install torchsample

!pip install git+https://github.com/ncullen93/torchsample
  
# Install gplearn

!pip install gplearn

  Cloning https://github.com/ncullen93/torchsample to /tmp/pip-req-build-m397gopi
  Running setup.py bdist_wheel for torchsample ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-9s1grbov/wheels/88/c7/72/14cd9a173eed1e29d0b17d866e7d9ee511d31a834aedd27489
Successfully built torchsample


In [8]:
!unzip \*.zip

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n

Archive:  embeddings.zip
replace glove.840B.300d/glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wiki-news-300d-1M/wiki-news-300d-1M.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace paragram_300_sl999/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace paragram_300_sl999/paragram_300_sl999.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n

4 archives were successfully processed.


In [9]:
import numpy as np
import pandas as pd 

import keras
from keras.utils import to_categorical
from keras.preprocessing import sequence

import torch
from torch import LongTensor, FloatTensor, no_grad
from torch.backends import cudnn
from torch.nn import Module, Sequential, Linear, Sigmoid, ReLU, Dropout
from torch.backends import cudnn
from torch.utils import checkpoint
from torch.optim import Adam

import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertForSequenceClassification, BertTokenizer

import gplearn
from gplearn.genetic import SymbolicTransformer

import torchsample
from torchsample.modules import ModuleTrainer

import os
import gc

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
train_data_df = pd.read_csv('train.csv')
train_data = train_data_df.values

In [0]:
targets = []

for i in range(0, len(train_data)):
    targets.append(train_data[i][2])
    
targets = np.array(targets)

In [0]:
sentences = []

for i in range(0, len(train_data)):
    sentences.append(train_data[i][1])

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

bag_of_words = []

for i in range(0, len(sentences)):
    bag_of_words.append(tokenizer.tokenize(sentences[i]))

In [0]:
sequences = []

for i in range(0, len(bag_of_words)):
    sequences.append(tokenizer.convert_tokens_to_ids(bag_of_words[i]))

In [0]:
# import itertools
# vocab  = set(list(itertools.chain(*bag_of_words)))

In [0]:
# toxic_questions = list(train_data_df.loc[train_data_df['target'] == 1]['question_text'])
# non_toxic_questions = list(train_data_df.loc[train_data_df['target'] == 0]['question_text'])

In [0]:
# def toxicity(word):
    # ct_toxic = 0

    # for question in toxic_questions:
        # if word in question:
            # ct_toxic = ct_toxic + 1
        
    # ct_non_toxic = 0

    # for question in non_toxic_questions:
        # if word in question:
            # ct_non_toxic = ct_non_toxic + 1
            
    # return (ct_toxic/len(toxic_questions)) / ((ct_non_toxic/len(non_toxic_questions)) + 0.0000001)

In [0]:
# toxicity_indices = dict()

# for word in vocab:
    # toxicity_indices[word] = toxicity(word)

In [0]:
# toxicities = []

# for question in bag_of_words:
    # question_toxicities = []
    
    # for word in question:
        # question_toxicities.append(toxicity_indices[word])
        
    # toxicities.append(question_toxicities)

In [0]:
# padded_toxicities = sequence.pad_sequences(toxicities, maxlen=100)

In [0]:
# function_set = ['add', 'sub', 'mul', 'div',
                # 'sqrt', 'log', 'abs', 'neg', 'inv',
                # 'max', 'min']

# gp = SymbolicTransformer(generations=10, population_size=2000,
                         # hall_of_fame=100, n_components=10,
                         # function_set=function_set,
                         # parsimony_coefficient=0.0005,
                         # max_samples=0.9, verbose=1,
                         # random_state=0, n_jobs=3)

# gp.fit(padded_toxicities, targets)

In [0]:
# engineered_features = gp._programs

# for i in range(len(engineered_features)):
    # for engineered_feature in engineered_features[i]:
        # if engineered_feature != None:
            # print(engineered_feature)

In [0]:
MAX_SEQUENCE_LENGTH = 128
TRAIN_VAL_SPLIT = 0.8
NUM_TEXT_FEATURES = 768

In [0]:
sequences = LongTensor(sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH))
# toxicities = LongTensor(toxicities)
targets = FloatTensor(targets.reshape(len(targets), 1))

In [0]:
train_sequences = sequences[:np.int32(TRAIN_VAL_SPLIT*len(sequences))]
# train_toxicities = toxicities[:np.int32(TRAIN_VAL_SPLIT*len(sequences))]
train_targets = targets[:np.int32(TRAIN_VAL_SPLIT*len(targets))]

val_sequences = sequences[np.int32(TRAIN_VAL_SPLIT*len(sequences)):]
# val_toxicities = toxicities[np.int32(TRAIN_VAL_SPLIT*len(sequences)):]
val_targets = targets[np.int32(TRAIN_VAL_SPLIT*len(targets)):]

In [0]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()

In [27]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterme

In [0]:
for param in model.parameters():
  param.requires_grad = False
  
for param in model.bert.encoder.layer[0].parameters():
  param.requires_grad = True
  
for param in model.bert.encoder.layer[11].parameters():
  param.requires_grad = True
  
for param in model.bert.pooler.parameters():
  param.requires_grad = True

In [0]:
model.classifier = Sequential(Linear(in_features=NUM_TEXT_FEATURES, out_features=100), ReLU(), Linear(in_features=100, out_features=20, bias=True), ReLU(), Linear(in_features=20, out_features=1, bias=True), Sigmoid()).cuda()

In [0]:
cudnn.benchmark = True

In [0]:
# train_text_features = []
# val_text_features = []

# print('Extract text features for training data')
# for i in range(len(train_sequences) - BATCH_SIZE):
  # train_batch = train_sequences[i : i + BATCH_SIZE].cuda()
  
  # with no_grad():
    # batch_text_features = model(train_batch)[-1]
    # train_text_features.extend(list(batch_text_features))
    # del batch_text_features
    # gc.collect()
  
  # print("Batch No. : " + str(i))
  # os.system('clear')

# print('Extract text features for validation data')
# for i in range(len(val_sequences) - BATCH_SIZE):
  # val_batch = val_sequences[i : i + BATCH_SIZE].cuda()
  
  # with no_grad():
    # batch_text_features = model(val_batch)[-1]
    # val_text_features.extend(list(batch_text_features))
    # del batch_text_features
    # gc.collect()
    
  # print("Batch No. : " + str(i))
  # os.system('clear')
     
# train_text_features = LongTensor(np.array(train_text_features))
# val_text_features = LongTensor(np.array(val_text_features))

In [0]:
     
# train_text_features = LongTensor(train_text_features)
# val_text_features = LongTensor(np.array(val_text_features))

In [0]:
trainer = ModuleTrainer(model)

In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
NUM_EPOCHS = 2

In [0]:
# GOOD BATCH SIZE + LEARNING RATE COMINATIONS :
# 1) 32 AND 0.00001
# 2) 

In [41]:
trainer.compile(loss='binary_cross_entropy',
                optimizer=Adam(params=model.parameters(), lr=LEARNING_RATE))

trainer.fit(train_sequences.cuda(), train_targets.cuda(),
            val_data=(val_sequences.cuda(), val_targets.cuda()),
            num_epoch=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=1)

Epoch 1/2:   0%|          | 0/32654 [00:00<?, ? batches/s]/usr/local/lib/python3.6/dist-packages/torchsample/modules/module_trainer.py:280: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  batch_logs['loss'] = loss.data[0]
Epoch 1/2:   0%|          | 2/32654 [00:01<5:29:34,  1.65 batches/s, loss=0.2450] 


RuntimeError: ignored